In [1]:
%load_ext autoreload 
%autoreload 2
%matplotlib inline 

In [2]:
import sys
sys.path.append('../../')

In [3]:
import math
import torch
from torch import nn
from torch.nn import functional as F

In [4]:
from omd2l.data import TimeMachine


In [5]:
from omd2l.models.base import Trainer


In [6]:
from omd2l.models.RNN import RNNScratch
from omd2l.models.RNN import RNNLMScratch
from omd2l.models.RNN import RNN
from omd2l.models.RNN import RNNLM

In [7]:
from omd2l.models.RNN import LSTMScratch
from omd2l.models.RNN import LSTM

In [8]:
from omd2l.models.RNN import GRUScratch
from omd2l.models.RNN import GRU

In [9]:
from omd2l.models.RNN import StackedRNNScratch


In [10]:
from omd2l.utils.compute import try_gpu

In [11]:
batch_size, num_inputs, num_hiddens, num_steps = 2, 16, 32, 100
rnn = RNNScratch(num_inputs, num_hiddens)
X = torch.ones((num_steps, batch_size, num_inputs))
outputs, state = rnn(X)

In [12]:
def check_len(a, n):  #@save
    """Check the length of a list."""
    assert len(a) == n, f'list\'s length {len(a)} != expected length {n}'

def check_shape(a, shape):  #@save
    """Check the shape of a tensor."""
    assert a.shape == shape, \
            f'tensor\'s shape {a.shape} != expected shape {shape}'

check_len(outputs, num_steps)
check_shape(outputs[0], (batch_size, num_hiddens))
check_shape(state, (batch_size, num_hiddens))

## RNN

### From Scratch 

In [13]:
model = RNNLMScratch(rnn, num_inputs)
outputs = model(torch.ones((batch_size, num_steps), dtype=torch.int64))
check_shape(outputs, (batch_size, num_steps, num_inputs))

TypeError: 'module' object is not callable

In [ ]:
data = TimeMachine(batch_size=1024, num_steps=32)
rnn = RNNScratch(num_inputs=len(data.vocab), num_hiddens=32)
model = RNNLMScratch(rnn, vocab_size=len(data.vocab), lr=1)
trainer = Trainer(max_epochs=100, gradient_clip_val=1, num_gpus=1)
trainer.fit(model, data)

In [ ]:
model.predict('it has', 20, data.vocab, try_gpu())

In [ ]:
model.predict('what is your', 20, data.vocab, try_gpu())

### Consice implementation 

In [ ]:
rnn = RNN(num_inputs=len(data.vocab), num_hiddens=32)
model = RNNLM(rnn, vocab_size=len(data.vocab), lr=1)
model.predict('it has', 20, data.vocab)

In [ ]:
trainer = Trainer(max_epochs=100, gradient_clip_val=1, num_gpus=1)
trainer.fit(model, data)

In [ ]:
model.predict('it has', 20, data.vocab, try_gpu())

## LSTM

In [ ]:
lstm = LSTMScratch(num_inputs=len(data.vocab), num_hiddens=32)
model = RNNLMScratch(lstm, vocab_size=len(data.vocab), lr=4)
trainer = Trainer(max_epochs=50, gradient_clip_val=1, num_gpus=1)
trainer.fit(model, data)

In [ ]:
lstm = LSTM(num_inputs=len(data.vocab), num_hiddens=32)
model = RNNLM(lstm, vocab_size=len(data.vocab), lr=4)
trainer.fit(model, data)

In [ ]:
model.predict('it has', 20, data.vocab, try_gpu())

## GRU

In [ ]:
data = TimeMachine(batch_size=1024, num_steps=32)
gru = GRUScratch(num_inputs=len(data.vocab), num_hiddens=32)
model = RNNLMScratch(gru, vocab_size=len(data.vocab), lr=4)
trainer = Trainer(max_epochs=50, gradient_clip_val=1, num_gpus=1)
trainer.fit(model, data)

In [ ]:
gru = GRU(num_inputs=len(data.vocab), num_hiddens=32, num_layers=1)
model = RNNLM(gru, vocab_size=len(data.vocab), lr=4)
trainer.fit(model, data)

In [ ]:
model.predict('it has', 20, data.vocab, try_gpu())

## Deep RNN

In [ ]:
rnn_block = StackedRNNScratch(num_inputs=len(data.vocab),
                              num_hiddens=32, num_layers=2)
model = RNNLMScratch(rnn_block, vocab_size=len(data.vocab), lr=2)
trainer = Trainer(max_epochs=100, gradient_clip_val=1, num_gpus=1)
trainer.fit(model, data)

In [ ]:
model.predict('it has', 20, data.vocab, try_gpu())

In [ ]:
gru = GRU(num_inputs=len(data.vocab), num_hiddens=32, num_layers=2)
model = RNNLM(gru, vocab_size=len(data.vocab), lr=2)
trainer.fit(model, data)

In [ ]:
model.predict('it has', 20, data.vocab, try_gpu())

In [ ]:
model.predict('time machine', 20, data.vocab, try_gpu())